In [1]:
import pandas as pd

df = pd.read_csv('FAOSTAT_data_12-5-2020.csv')
print(df.head(n = 3))
print("\n",df.tail(n = 2))


  Domain Code Domain  Area Code       Area  Element Code Element  Item Code  \
0          QC  Crops          9  Argentina          5419   Yield        515   
1          QC  Crops          9  Argentina          5419   Yield        515   
2          QC  Crops          9  Argentina          5419   Yield        515   

     Item  Year Code  Year   Unit   Value Flag Flag Description  
0  Apples       2016  2016  hg/ha  223912   Fc  Calculated data  
1  Apples       2017  2017  hg/ha  235386   Fc  Calculated data  
2  Apples       2018  2018  hg/ha  213072   Fc  Calculated data  

    Domain Code Domain  Area Code                      Area  Element Code  \
64          QC  Crops        231  United States of America          5510   
65          QC  Crops        231  United States of America          5510   

       Element  Item Code      Item  Year Code  Year    Unit     Value Flag  \
64  Production        388  Tomatoes       2017  2017  tonnes  11141862  NaN   
65  Production        388  Tom

In [3]:
# DataFrame columns and index
print(df.columns, df.index)

# DataFrame shape
print('\n', df.shape)

# data type in each column
print('\n', df.dtypes)

Index(['Domain Code', 'Domain', 'Area Code', 'Area', 'Element Code', 'Element',
       'Item Code', 'Item', 'Year Code', 'Year', 'Unit', 'Value', 'Flag',
       'Flag Description'],
      dtype='object') RangeIndex(start=0, stop=66, step=1)

 (66, 14)

 Domain Code         object
Domain              object
Area Code            int64
Area                object
Element Code         int64
Element             object
Item Code            int64
Item                object
Year Code            int64
Year                 int64
Unit                object
Value                int64
Flag                object
Flag Description    object
dtype: object


In [5]:
# dataset information
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66 entries, 0 to 65
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Domain Code       66 non-null     object
 1   Domain            66 non-null     object
 2   Area Code         66 non-null     int64 
 3   Area              66 non-null     object
 4   Element Code      66 non-null     int64 
 5   Element           66 non-null     object
 6   Item Code         66 non-null     int64 
 7   Item              66 non-null     object
 8   Year Code         66 non-null     int64 
 9   Year              66 non-null     int64 
 10  Unit              66 non-null     object
 11  Value             66 non-null     int64 
 12  Flag              43 non-null     object
 13  Flag Description  66 non-null     object
dtypes: int64(6), object(8)
memory usage: 7.3+ KB
None


In [7]:
# unique values in the Area, Element and Item columns
print('Area:', df['Area'].unique())
print('\nElement:', df['Element'].unique())
print('\nItems:', df['Item'].unique())


Area: ['Argentina' 'Brazil' 'United States of America']

Element: ['Yield' 'Production']

Items: ['Apples' 'Bananas' 'Cherries' 'Tomatoes']


In [8]:
# Remove the columns: 'Domain Code', 'Domain', 'Area Code', 'Element Code', 'Item Code', 'Year Code', 'Flag', 'Flag Description'
col = ['Domain Code', 'Domain', 'Area Code', 'Element Code', 'Item Code', 'Year Code', 'Flag', 'Flag Description']
df.drop(columns = col, axis = 1, inplace = True)
print(df.head(n = 2))

        Area Element    Item  Year   Unit   Value
0  Argentina   Yield  Apples  2016  hg/ha  223912
1  Argentina   Yield  Apples  2017  hg/ha  235386


In [9]:
# converts the name of each country into their respective acronyms
def edit_countries_name(country):
    if country == 'Argentina':
        return 'AR'
    elif country == 'United States of America':
        return 'USA'
    else:
        return 'BR'
df['Area'] = df['Area'].apply(edit_countries_name)

# Convert the value column dtype to float
df['Value'] = df['Value'].astype(float)

# Create a new column called Country / Commodity
df['Country/Commodity'] = df['Area']+' '+df['Item']

# Group the data into production and productivity
production = df.groupby('Element').get_group('Production')
yield_ = df.groupby('Element').get_group('Yield')

# Change the shape of the dataframes with the pivot function, so that the new objects have a 'Year' index, 
# a 'Country / Commodity' column and the values correspond to a 'Value' column
production = production.pivot(index = 'Year', columns = 'Country/Commodity', values = 'Value')
yield_ = yield_.pivot(index = 'Year', columns = 'Country/Commodity', values = 'Value')
print(production,'\n')
print(yield_)


Country/Commodity  AR Apples  AR Bananas  AR Cherries  AR Tomatoes  BR Apples  \
Year                                                                            
2016                458908.0    174644.0       7207.0     658237.0  1055383.0   
2017                458908.0    174615.0       7246.0     655861.0  1307642.0   
2018                510478.0    174585.0       7285.0     653485.0  1195007.0   

Country/Commodity  BR Bananas  BR Tomatoes  USA Apples  USA Bananas  \
Year                                                                  
2016                6625211.0    4166789.0   5214040.0       3387.0   
2017                6584967.0    4225414.0   5240670.0       3992.0   
2018                6752171.0    4110242.0   4652500.0       4274.0   

Country/Commodity  USA Cherries  USA Tomatoes  
Year                                           
2016                   315454.0    12877049.0  
2017                   398140.0    11141862.0  
2018                   312430.0    12612139.0 

In [10]:
# Concatenate the objects created in the previous exercise
df_mod = pd.concat([production, yield_], axis = 0, keys = ['Production', 'Yield'])
print(df_mod)

Country/Commodity  AR Apples  AR Bananas  AR Cherries  AR Tomatoes  BR Apples  \
           Year                                                                 
Production 2016     458908.0    174644.0       7207.0     658237.0  1055383.0   
           2017     458908.0    174615.0       7246.0     655861.0  1307642.0   
           2018     510478.0    174585.0       7285.0     653485.0  1195007.0   
Yield      2016     223912.0    209247.0      28417.0     393807.0   308465.0   
           2017     235386.0    209343.0      30220.0     394319.0   392355.0   
           2018     213072.0    209440.0      29790.0     394832.0   361805.0   

Country/Commodity  BR Bananas  BR Tomatoes  USA Apples  USA Bananas  \
           Year                                                       
Production 2016     6625211.0    4166789.0   5214040.0       3387.0   
           2017     6584967.0    4225414.0   5240670.0       3992.0   
           2018     6752171.0    4110242.0   4652500.0       4274.0